### Tools

In [ ]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
wiki_api_wrapper = WikipediaAPIWrapper(doc_content_chars_max=500,top_k_results=1)
wiki_tools = WikipediaQueryRun(api_wrapper=wiki_api_wrapper)

In [ ]:
from langchain_community.tools import ArxivQueryRun
from langchain_community.utilities import ArxivAPIWrapper
arxiv_api_wrapper = ArxivAPIWrapper(doc_content_chars_max=500,top_k_results=1)
arxiv_tool = ArxivQueryRun(api_wrapper=arxiv_api_wrapper)

In [ ]:
# Custom tool 
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import FAISS


loader = WebBaseLoader(web_paths=["https://python.langchain.com/docs/"])
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size =1000,chunk_overlap = 200)
splitted_documents = text_splitter.split_documents(docs)
embedding = OllamaEmbeddings(model="llama3.2")
vector_db = FAISS.from_documents(splitted_documents,embedding)
retriever = vector_db.as_retriever()

In [ ]:
from langchain.tools.retriever import create_retriever_tool
langChain_tool = create_retriever_tool(retriever=retriever,name="langChian",description="langchain search tool")

In [ ]:
tools = [wiki_tools,arxiv_tool]
tools

### Agents 

In [ ]:
from langchain_groq import ChatGroq
llm = ChatGroq(model="llama-3.2-11b-vision-preview",)

In [ ]:
from langchain.prompts import  ChatPromptTemplate
from langchain.prompts import  PromptTemplate
from langchain.prompts import SystemMessagePromptTemplate
from langchain.prompts import MessagesPlaceholder
from langchain.prompts import HumanMessagePromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
    MessagesPlaceholder(variable_name='chat_history', optional=True),
    HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
    MessagesPlaceholder(variable_name='agent_scratchpad')
    ]
)

In [ ]:
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")


In [ ]:
model_with_tools = llm.bind_tools(tools)

In [ ]:
from langchain.agents import create_tool_calling_agent
agent = create_tool_calling_agent(llm, tools, prompt)

In [ ]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools,verbose=True,max_iterations=2)

In [ ]:
agent_executor.invoke({"input": "what is machine learning"})